In [ ]:
from dotenv import load_dotenv
import logging
import numpy as np
import dataUtil as DU

In [ ]:
import pandas as pd
import datetime as dt
import os 
import json 
import warnings

warnings.filterwarnings("ignore")

In [ ]:
def convert_toppath(topR, basep):
    # print(topR)
    obj=topR.Objective
    rm=topR.Risk_measure
    intv=topR.R_Interval
    rb_filep = os.path.join(basep, f"weights_{obj}_{intv}_{rm}.csv")
    print(rb_filep)
    return rb_filep, obj, intv, rm

In [ ]:
load_dotenv() #Check path for env variables
logging.getLogger().setLevel(logging.DEBUG)

In [ ]:
# filenames = next(walk(data_path), (None, None, []))[2]  # [] if no file
rebalance_date = dt.datetime(2025, 2, 8)
data_path = rebalance_date.strftime("Rebalance-%Y%m%d")
rb_folders = [f for f in os.listdir(data_path) if ("Classic_" in f) and os.path.isdir(os.path.join(data_path, f))]
display(rb_folders)

In [ ]:
for rb_folder in rb_folders:
    display(rb_folder)
    assets_selection =  rb_folder.split('_')[1]
    int_v =  list( rb_folder.split('_')[3])
    print(assets_selection, int_v)
    for i in int_v:
        outputp = os.path.join(data_path, rb_folder)
        display(outputp)
        RB_metric = pd.read_csv(os.path.join(outputp, "Port_Metric.csv")).dropna()
        # display(RB_metric)
        for itv in ['MinRisk','Sharpe','MaxRet']:
            print(i, itv)
            mask1 = (RB_metric['R_Interval'].isin([i]))
            mask2 = (RB_metric['Objective'].isin([itv]))
            metric = RB_metric.loc[mask1 & mask2].sort_values(by=['CAGR'], ascending=False)
            display(metric.head(10))
            # metric = RB_metric.loc[(RB_metric['R_Interval']==i) & (RB_metric['Objective']==itv)].sort_values(by=['CAGR', 'Sharpe Ratio'], ascending=False)
            top_file, obj, intv, rm = convert_toppath(metric.iloc[0], outputp)
    
            we = pd.read_csv(top_file).fillna(0.0)
            display(we.tail())
            i_data = we.iloc[-1].Date
            print(i_data)
            w_dict = json.dumps(we.drop(columns=['index','Date']).iloc[-1].to_dict())
            print(w_dict)
    
            sql_s = f"INSERT INTO Trading.weights (date, port_name, risk_mes, obj_type, int_type, weights) VALUES (\'{i_data}\', \'{assets_selection}\', \'{rm}\', \'{obj}\', \'{intv}\', \'{w_dict}\');"
            display(sql_s)
    
            DU.ExecSQL(sql_s)